# XG Boost

In [21]:
import numpy as np 
import pandas as pd 
import xgboost as xgb 
import sklearn
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    average_precision_score,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
    roc_auc_score,
    precision_recall_curve,
    auc
)

In [5]:
X_train = pd.read_csv('../Data/f2_data/f2_X_train.csv')
y_train = pd.read_csv('../Data/f2_data/f2_y_train.csv')

X_val = pd.read_csv('../Data/f2_data/f2_X_val.csv')
y_val = pd.read_csv('../Data/f2_data/f2_y_val.csv')

X_test = pd.read_csv('../Data/f2_data/f2_X_test.csv')  
y_test = pd.read_csv('../Data/f2_data/f2_y_test.csv')

In [13]:
X_train.head()

,dwell_0,dwell_1,dwell_2,log_mean_0,log_mean_1,log_mean_2,log_std_0,log_std_1,log_std_2,base_1_A,...,base_2_T,base_3_A,base_3_G,base_6_A,base_6_C,base_6_T,base_7_A,base_7_C,base_7_G,base_7_T
0,119.626866,126.970149,81.335821,-4.880373,-4.760620,-5.009353,1.589266,1.923420,1.406697,1,...,0,0,1,0,1,0,1,0,0,0
1,103.245333,123.093333,82.341333,-4.693492,-5.071426,-5.112690,1.305410,1.935764,1.570337,0,...,1,0,1,0,1,0,0,1,0,0
2,101.653810,96.965163,86.640435,-4.710294,-4.959351,-5.066744,0.744468,1.010792,0.930151,0,...,0,1,0,1,0,0,1,0,0,0
3,92.394118,92.967647,84.441176,-5.061619,-5.050722,-4.853481,0.708905,0.852047,0.672494,0,...,1,1,0,0,1,0,0,1,0,0
4,106.682025,121.106329,86.660000,-5.034657,-4.870927,-4.946987,1.199370,1.795066,1.223444,0,...,1,0,1,1,0,0,1,0,0,0


## SMOTE

In [22]:
# Check class distribution before SMOTE
unique, counts = np.unique(y_train, return_counts=True)
print(f"\nOriginal training data distribution:")
print(f"Class 0: {counts[0]} ({counts[0]/len(y_train)*100:.2f}%)")
print(f"Class 1: {counts[1]} ({counts[1]/len(y_train)*100:.2f}%)")

# Apply SMOTE
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Check class distribution after SMOTE
unique, counts = np.unique(y_train_balanced, return_counts=True)
print(f"\nBalanced training data distribution:")
print(f"Class 0: {counts[0]} ({counts[0]/len(y_train_balanced)*100:.2f}%)")
print(f"Class 1: {counts[1]} ({counts[1]/len(y_train_balanced)*100:.2f}%)")


Original training data distribution:
Class 0: 83628 (95.33%)
Class 1: 4095 (4.67%)

Balanced training data distribution:
Class 0: 83628 (50.00%)
Class 1: 83628 (50.00%)


### Train baseline model

In [26]:
# train baseline model
baseline_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='aucpr',
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

baseline_model.fit(
    X_train_balanced, y_train_balanced,
    eval_set=[(X_val, y_val)],
    verbose=False
)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'aucpr'


### helper function to evaluate metrics

In [29]:
def evaluate_model(model, X, y, dataset_name):
    """Evaluate model and return comprehensive metrics"""
    y_pred_proba = model.predict_proba(X)[:, 1]
    y_pred = model.predict(X)
    
    # Calculate confusion matrix
    cm = confusion_matrix(y, y_pred)
    
    # Calculate metrics using sklearn functions
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    accuracy = accuracy_score(y, y_pred)
    roc_auc = roc_auc_score(y, y_pred_proba)
    
    # Calculate PR-AUC
    precision_vals, recall_vals, _ = precision_recall_curve(y, y_pred_proba)
    pr_auc = auc(recall_vals, precision_vals)
    
    # Get confusion matrix components
    tn, fp, fn, tp = cm.ravel()
    
    print(f"{dataset_name} Set Metrics")
    
    print("Confusion Matrix:\n", cm)    
    print(f"  Precision:  {precision:.4f}")
    print(f"  Recall:     {recall:.4f}")
    print(f"  F1-Score:   {f1:.4f}")
    print(f"  Accuracy:   {accuracy:.4f}")
    print(f"  ROC-AUC:    {roc_auc:.4f}")
    print(f"  PR-AUC:     {pr_auc:.4f}")
    
    return {
        'pr_auc': pr_auc,
        'roc_auc': roc_auc,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tp': tp,
        'y_pred_proba': y_pred_proba,
        'y_pred': y_pred,
        'precision_curve': precision_vals,
        'recall_curve': recall_vals
    }

baseline_val_results = evaluate_model(baseline_model, X_val, y_val, "Validation")
baseline_test_results = evaluate_model(baseline_model, X_test, y_test, "Test")

Validation Set Metrics
Confusion Matrix:
 [[19006  1941]
 [  307   677]]
  Precision:  0.2586
  Recall:     0.6880
  F1-Score:   0.3759
  Accuracy:   0.8975
  ROC-AUC:    0.8953
  PR-AUC:     0.4062
Test Set Metrics
Confusion Matrix:
 [[10734  1054]
 [  123   273]]
  Precision:  0.2057
  Recall:     0.6894
  F1-Score:   0.3169
  Accuracy:   0.9034
  ROC-AUC:    0.8965
  PR-AUC:     0.3505


### Hyperparameter Tuning

In [32]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for hyperparameter tuning
param_grid = {
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Create base model for tuning
xgb_base = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='aucpr',
    scale_pos_weight=1,  # Classes balanced by SMOTE
    random_state=42,
    tree_method='gpu_hist'
)

grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    scoring='average_precision',  # PR-AUC
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train_balanced, y_train_balanced)
print("Best params:", grid_search.best_params_)
    

Fitting 3 folds for each of 2187 candidates, totalling 6561 fits


ValueError: 
All the 6561 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jinji\Documents\NUS\DSA4262\project\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jinji\Documents\NUS\DSA4262\project\.venv\lib\site-packages\xgboost\core.py", line 729, in inner_f
    return func(**kwargs)
  File "c:\Users\jinji\Documents\NUS\DSA4262\project\.venv\lib\site-packages\xgboost\sklearn.py", line 1683, in fit
    self._Booster = train(
  File "c:\Users\jinji\Documents\NUS\DSA4262\project\.venv\lib\site-packages\xgboost\core.py", line 729, in inner_f
    return func(**kwargs)
  File "c:\Users\jinji\

### Train final model with best parameters

In [ ]:
final_model = xgb.XGBClassifier(
    **best_params,
    objective='binary:logistic',
    eval_metric='aucpr',
    scale_pos_weight=1,  # Classes balanced by SMOTE
    random_state=42,
    tree_method='hist'
)

# Train final model
final_model.fit(
    X_train_balanced, y_train_balanced,
    eval_set=[(X_val, y_val)],
    verbose=False
)

# Evaluate final model
final_val_results = evaluate_model(final_model, X_val, y_val, "Validation")
final_test_results = evaluate_model(final_model, X_test, y_test, "Test")